In [ ]:
"""
Author: Bryan Bo Cao
Email: boca7588@colorado.edu or bo.cao-1@colorado.edu
Github Repo: https://github.com/BryanBo-Cao/neuralnets-deeplearning
Reference:
    http://www.deeplearning.net/tutorial/lstm.html#lstm
    https://github.com/llSourcell/LSTM_Networks/blob/master/LSTM%20Demo.ipynb
    https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/recurrent_network.py
    Recurrent Neural Network.
    A Recurrent Neural Network (LSTM) implementation example using TensorFlow library.
    This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
    Links:
    [Long Short Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf)
    [MNIST Dataset](http://yann.lecun.com/exdb/mnist/).
    Author: Aymeric Damien
    Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import random
import copy
import matplotlib.pyplot as plt
from importlib import import_module

H = 25
N = 5

#Reference: Denis
def generate_parity_sequences(N, count):
    """
    Generate :count: sequences of length :N:.
    If odd # of 1's -> output 1
    else -> output 0
    """
    xor = lambda x: 1 if (x % 2 == 1) else 0
    sequences = np.random.choice([0, 1], size=[count, N], replace=True)
    counts = np.count_nonzero(sequences == 1, axis=1)
    # xor each sequence, expand dimensions by 1 to match sequences shape
    y = np.expand_dims(np.array([xor(x) for x in counts]), axis=1)

    # In case if you wanted to have the answer just appended at the end of the sequence:
    #     # append the answer at the end of each sequence
    #     seq_plus_y = np.concatenate([sequences, y], axis=1)
    #     print(sequences.shape, y.shape, seq_plus_y.shape)
    #     return seq_plus_y
    return np.expand_dims(sequences, axis=2), y

#Reference: Modified from Denis by Bo Cao
def generate_parity_sequences(N, count):
    """
    Generate :count: sequences of length :N:.
    If odd # of 1's -> output 1
    else -> output 0
    """
    xor = lambda x: 1 if (x % 2 == 1) else 0
    sequences = np.random.choice([0, 1], size=[count, N], replace=True)
    counts = np.count_nonzero(sequences == 1, axis=1)
    # xor each sequence, expand dimensions by 1 to match sequences shape
    y = np.expand_dims(np.array([xor(x) for x in counts]), axis=1)

    # In case if you wanted to have the answer just appended at the end of the sequence:
    #     # append the answer at the end of each sequence
    #     seq_plus_y = np.concatenate([sequences, y], axis=1)
    #     print(sequences.shape, y.shape, seq_plus_y.shape)
    #     return seq_plus_y
    
    new_y = []
    for i in range(len(y)):
        new_yy = []
        if y[i] == 0:
            new_yy.append(0)
            new_yy.append(1)
        else:
            new_yy.append(1)
            new_yy.append(0)
        new_y.append(new_yy)

    return np.expand_dims(sequences, axis=2), new_y

In [ ]:
# Training Parameters
learning_rate = 0.25
batch_size = 128
display_step = batch_size * 100

#batch_steps = 10000 / batch_size
epochs = 3000

# Network Parameters
num_input = 1 # 
timesteps = N # timesteps
num_hidden = H # hidden layer num of features
num_classes = 2 # 0 or 1

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

logits = RNN(X, weights, biases)
#prediction = tf.nn.softmax(logits)
prediction = tf.tanh(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

test_accuracies_10replications = []
losses_1st_replication = [] #epoch as unit
test_accuracies_1st_replication = [] #epoch as unit
train_accuracies_1st_replication = [] #epoch as unit

# Start training
with tf.Session() as sess:
    
    # Run 10 replications
    for replication in range(10):
        
        print("Replication: %d: " % replication)
        
        # Initialize random weights
        train_data = generate_parity_sequences(N, 10000)
        train_data_x = train_data[0]
        train_data_y = train_data[1]
        test_data = generate_parity_sequences(N, 10000)
        test_data_x = test_data[0]
        test_data_y = test_data[1]
        
        # Run the initializer
        sess.run(init)
    
        for epoch in range(epochs):
            batch_index = 0
            while batch_index < 10000:

                train_data_batch_x = []
                train_data_batch_y = []
                if batch_index + batch_size < 10000: 
                    train_data_batch_x = train_data_x[batch_index : batch_index + batch_size]
                    train_data_batch_y = train_data_y[batch_index : batch_index + batch_size]
                else:
                    train_data_batch_x = train_data_x[batch_index : ]
                    train_data_batch_y = train_data_y[batch_index : ]

                #batch_x, batch_y = mnist.train.next_batch(batch_size)
                # Reshape data to get 28 seq of 28 elements
                #batch_x = batch_x.reshape((batch_size, timesteps, num_input))
                #train_data_x = train_data_x.reshape((10000, timesteps, num_input))
                #print("train_data_batch_x.shape:  " , train_data_batch_x.shape)
                if batch_index + batch_size < 10000: 
                    train_data_batch_x = train_data_batch_x.reshape((batch_size, timesteps, num_input))
                else:
                    train_data_batch_x = train_data_batch_x.reshape((10000 % batch_size, timesteps, num_input))
                # Run optimization op (backprop)
                #sess.run(train_op, feed_dict={X: train_data_x, 
                 #                             Y: train_data_y})
                sess.run(train_op, feed_dict={X: train_data_batch_x, 
                                              Y: train_data_batch_y})

                batch_index += batch_size

            if replication == 0:
                loss, train_accuracy = sess.run([loss_op, accuracy], feed_dict={X: train_data_x, Y: train_data_y})
                test_accuracy = sess.run(accuracy, feed_dict={X: test_data_x, Y: test_data_y})
                losses_1st_replication.append(loss)
                train_accuracies_1st_replication.append(train_accuracy)
                test_accuracies_1st_replication.append(test_accuracy)
            
            if epoch % 10 == 0:
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: train_data_x,
                                                                         Y: train_data_y})
                print("Epoch: " + str(epoch) + \
                          ", Loss= " + \
                          "{:.4f}".format(loss) + ", Training Accuracy= " + \
                          "{:.3f}".format(acc))

        print("Optimization Finished!")

        test_data_x = test_data_x.reshape((-1, timesteps, num_input))
        test_data_y = test_data_y
        test_accuracy = sess.run(accuracy, feed_dict={X: test_data_x, Y: test_data_y})
        test_accuracies_10replications.append(test_accuracy)
        print("Testing Accuracy:", test_accuracy)

Replication: 0: 
Epoch: 0, Loss= 0.7141, Training Accuracy= 0.497
Epoch: 10, Loss= 0.6997, Training Accuracy= 0.497
Epoch: 20, Loss= 0.6985, Training Accuracy= 0.497
Epoch: 30, Loss= 0.6977, Training Accuracy= 0.497
Epoch: 40, Loss= 0.6971, Training Accuracy= 0.497
Epoch: 50, Loss= 0.6967, Training Accuracy= 0.497
Epoch: 60, Loss= 0.6963, Training Accuracy= 0.497
Epoch: 70, Loss= 0.6960, Training Accuracy= 0.497
Epoch: 80, Loss= 0.6957, Training Accuracy= 0.497
Epoch: 90, Loss= 0.6955, Training Accuracy= 0.497
Epoch: 100, Loss= 0.6953, Training Accuracy= 0.497
Epoch: 110, Loss= 0.6951, Training Accuracy= 0.497
Epoch: 120, Loss= 0.6950, Training Accuracy= 0.497
Epoch: 130, Loss= 0.6948, Training Accuracy= 0.497
Epoch: 140, Loss= 0.6947, Training Accuracy= 0.497
Epoch: 150, Loss= 0.6946, Training Accuracy= 0.497
Epoch: 160, Loss= 0.6945, Training Accuracy= 0.497
Epoch: 170, Loss= 0.6945, Training Accuracy= 0.497
Epoch: 180, Loss= 0.6944, Training Accuracy= 0.497
Epoch: 190, Loss= 0.6943,

Epoch: 1600, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1610, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1620, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1630, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1640, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1650, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1660, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1670, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1680, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1690, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1700, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1710, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1720, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1730, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1740, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1750, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1760, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1770, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1780, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1790,

Epoch: 170, Loss= 0.6933, Training Accuracy= 0.471
Epoch: 180, Loss= 0.6933, Training Accuracy= 0.442
Epoch: 190, Loss= 0.6933, Training Accuracy= 0.442
Epoch: 200, Loss= 0.6933, Training Accuracy= 0.442
Epoch: 210, Loss= 0.6933, Training Accuracy= 0.442
Epoch: 220, Loss= 0.6933, Training Accuracy= 0.442
Epoch: 230, Loss= 0.6933, Training Accuracy= 0.442
Epoch: 240, Loss= 0.6933, Training Accuracy= 0.474
Epoch: 250, Loss= 0.6932, Training Accuracy= 0.474
Epoch: 260, Loss= 0.6932, Training Accuracy= 0.474
Epoch: 270, Loss= 0.6932, Training Accuracy= 0.474
Epoch: 280, Loss= 0.6932, Training Accuracy= 0.474
Epoch: 290, Loss= 0.6932, Training Accuracy= 0.474
Epoch: 300, Loss= 0.6932, Training Accuracy= 0.474
Epoch: 310, Loss= 0.6932, Training Accuracy= 0.474
Epoch: 320, Loss= 0.6932, Training Accuracy= 0.474
Epoch: 330, Loss= 0.6932, Training Accuracy= 0.474
Epoch: 340, Loss= 0.6932, Training Accuracy= 0.474
Epoch: 350, Loss= 0.6932, Training Accuracy= 0.474
Epoch: 360, Loss= 0.6932, Train

Epoch: 1770, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 1780, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 1790, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 1800, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 1810, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 1820, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 1830, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 1840, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 1850, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 1860, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 1870, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 1880, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 1890, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 1900, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1910, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1920, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1930, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1940, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1950, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1960,

Epoch: 350, Loss= 0.6932, Training Accuracy= 0.476
Epoch: 360, Loss= 0.6931, Training Accuracy= 0.510
Epoch: 370, Loss= 0.6931, Training Accuracy= 0.510
Epoch: 380, Loss= 0.6931, Training Accuracy= 0.510
Epoch: 390, Loss= 0.6931, Training Accuracy= 0.510
Epoch: 400, Loss= 0.6931, Training Accuracy= 0.510
Epoch: 410, Loss= 0.6931, Training Accuracy= 0.510
Epoch: 420, Loss= 0.6931, Training Accuracy= 0.510
Epoch: 430, Loss= 0.6931, Training Accuracy= 0.510
Epoch: 440, Loss= 0.6931, Training Accuracy= 0.510
Epoch: 450, Loss= 0.6930, Training Accuracy= 0.510
Epoch: 460, Loss= 0.6930, Training Accuracy= 0.510
Epoch: 470, Loss= 0.6930, Training Accuracy= 0.510
Epoch: 480, Loss= 0.6930, Training Accuracy= 0.510
Epoch: 490, Loss= 0.6930, Training Accuracy= 0.510
Epoch: 500, Loss= 0.6930, Training Accuracy= 0.510
Epoch: 510, Loss= 0.6930, Training Accuracy= 0.510
Epoch: 520, Loss= 0.6930, Training Accuracy= 0.510
Epoch: 530, Loss= 0.6930, Training Accuracy= 0.510
Epoch: 540, Loss= 0.6930, Train

Epoch: 1940, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1950, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1960, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1970, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1980, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 1990, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2000, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2010, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2020, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2030, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2040, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2050, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2060, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2070, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2080, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2090, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2100, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2110, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2120, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2130,

Epoch: 520, Loss= 0.6949, Training Accuracy= 0.496
Epoch: 530, Loss= 0.6949, Training Accuracy= 0.496
Epoch: 540, Loss= 0.6949, Training Accuracy= 0.496
Epoch: 550, Loss= 0.6949, Training Accuracy= 0.496
Epoch: 560, Loss= 0.6949, Training Accuracy= 0.496
Epoch: 570, Loss= 0.6948, Training Accuracy= 0.496
Epoch: 580, Loss= 0.6948, Training Accuracy= 0.496
Epoch: 590, Loss= 0.6948, Training Accuracy= 0.496
Epoch: 600, Loss= 0.6948, Training Accuracy= 0.496
Epoch: 610, Loss= 0.6948, Training Accuracy= 0.496
Epoch: 620, Loss= 0.6948, Training Accuracy= 0.496
Epoch: 630, Loss= 0.6947, Training Accuracy= 0.496
Epoch: 640, Loss= 0.6947, Training Accuracy= 0.496
Epoch: 650, Loss= 0.6947, Training Accuracy= 0.496
Epoch: 660, Loss= 0.6947, Training Accuracy= 0.496
Epoch: 670, Loss= 0.6947, Training Accuracy= 0.496
Epoch: 680, Loss= 0.6947, Training Accuracy= 0.496
Epoch: 690, Loss= 0.6947, Training Accuracy= 0.496
Epoch: 700, Loss= 0.6946, Training Accuracy= 0.496
Epoch: 710, Loss= 0.6946, Train

Epoch: 2110, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2120, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2130, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2140, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2150, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2160, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2170, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2180, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2190, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2200, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2210, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2220, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2230, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2240, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2250, Loss= 0.0001, Training Accuracy= 1.000
Epoch: 2260, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2270, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2280, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2290, Loss= 0.0000, Training Accuracy= 1.000
Epoch: 2300,

In [ ]:
# print results
test_accuracies_10replications_std = np.std(test_accuracies_10replications, axis=0)
test_accuracies_10replications_std_mean = test_accuracies_10replications_std / np.square(10)
print("test_accuracies_10replications: ", test_accuracies_10replications)
print("mean of test_accuracies_10replications: ", np.mean(test_accuracies_10replications))
print("standard deviation of test_accuracies_10replications_std_mean: ", test_accuracies_10replications_std_mean)

In [ ]:
losses_1st_replication
plt.plot(losses_1st_replication, color='green', linewidth=5)
plt.plot(train_accuracies_1st_replication, color='blue', linewidth=7)
plt.plot(test_accuracies_1st_replication, color='red', linewidth=3)
plt.xlim(0, epochs)
plt.xlabel("epoch")
plt.ylim(0, 1.2)
plt.ylabel("train acc in blue, test acc in red, loss in green")
plt.title("1st replication")
plt.show()